In [1]:
import os
import numpy as np
import tensorflow as tf
from scipy.misc import imread, imresize
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def print_typeshape(img):
    print ("Type is %s" % (type(img)))
    print ("Shape is %s" % (img.shape,))

In [3]:
def dataList(pathName):
    data_img_name = []
    data_label = []
    label_list = []

    for file in os.listdir(pathName):
        label_list.append(file)

    # http://stackoverflow.com/questions/3964681/find-all-files-in-directory-with-extension-txt-in-python
    for i, label in enumerate(label_list):

#         print('>> ' + str(i) + ':' + label)
        for file2 in os.listdir(pathName + os.sep + label):
            data_img_name.append(pathName + os.sep + label + os.sep + file2)
            data_label.append(i)
            
    return data_img_name, data_label, label_list

In [4]:
root_path = "/data/nv_dlcontest_dataset/train_VGG"

data_img_name, data_label, label_list = dataList(root_path)

eye_ref = np.eye(len(label_list), len(label_list))

In [5]:
with open('/data/nv_dlcontest_dataset/train_VGG/apple_pie/100605') as f:
    weight = np.load(f)
    print weight.shape

(1, 32, 32, 512)


In [6]:
# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

img_x = 32
img_y = 32
img_color = 512
img_data = img_x*img_y*img_color

n_input = img_x*img_y*img_color
n_output = len(label_list)

In [7]:
def getTrainData(idxList, data_img_name, data_label):
    num_data = len(idxList)
    _img = np.ndarray(shape=(num_data, img_x, img_y, img_color), dtype=float)
    _label = np.ndarray(shape=(num_data, len(label_list)), dtype=int)
    
    row = 0
#     for i, idx in enumerate(idxList):
    for idx in idxList:

        # file loading
        tmp_img = None
        tmp_fname = data_img_name[idx]
        with open(tmp_fname) as f:
            tmp_img = np.load(f)
            
        if len(tmp_img.shape)<3:
            continue
        
        _img[row] = tmp_img
        _label[row] = eye_ref[data_label[idx], :]

        row += 1
        
    return _img[:row, :], _label[:row, :]

In [8]:
!ls /data/nv_dlcontest_dataset/train_VGG/

apple_pie	    escargots		     ojingeo_bokkeum
baby_back_ribs	    falafel		     omelette
baklava		    filet_mignon	     onion_rings
beef_carpaccio	    fish_and_chips	     oysters
beef_tartare	    foie_gras		     pad_thai
beet_salad	    french_fries	     paella
beignets	    french_onion_soup	     pajeon
bibimbap	    french_toast	     pancakes
bossam		    fried_calamari	     panna_cotta
bread_pudding	    fried_rice		     peking_duck
breakfast_burrito   frozen_yogurt	     pho
bruschetta	    galbijjim		     pizza
bulgogi		    galchijorim		     pork_chop
caesar_salad	    ganjang_gejang	     poutine
cannoli		    garlic_bread	     prime_rib
caprese_salad	    gimbob		     pulled_pork_sandwich
carrot_cake	    gnocchi		     ramen
ceviche		    greek_salad		     ravioli
cheese_plate	    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
chicken_curry	    guacamole		     samgupsal
chicken_quesadilla  gyoza		     samgyetang
chicken_wings	    hamburger		     samosa
cho

In [8]:
# Test
weights = {
    'wd1': tf.Variable(tf.random_normal([img_data, n_output], stddev=0.1)),
}
biases = {
    'bd1': tf.Variable(tf.random_normal([n_output], stddev=0.1)),
}

def conv_basic(_input, _w, _b, _keepratio):
    # Input
    _input_r = tf.reshape(_input, shape=[-1, img_y * img_x * img_color])
    
    # Fc1
    _out = tf.add(tf.matmul(_input_r, _w['wd1']), _b['bd1'])
    
    # Return everythin
    out = {
        'input_r': _input_r,
        'out': _out
    }
    
    return out

In [9]:
# weights = {
#     'wd1': tf.Variable(tf.random_normal([img_data, 2048], stddev=0.1)),
#     'wd2': tf.Variable(tf.random_normal([2048, n_output], stddev=0.1)),
# }
# biases = {
#     'bd1': tf.Variable(tf.random_normal([2048], stddev=0.1)),
#     'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1)),
# }

# def conv_basic(_input, _w, _b, _keepratio):
#     # Input
#     _input_r = tf.reshape(_input, shape=[-1, img_y * img_x * img_color])
    
#     # Fc1
#     _fc1 = tf.nn.relu(tf.add(tf.matmul(_input_r, _w['wd1']), _b['bd1']))
#     _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    
#     # Fc2
#     _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    
#     # Return everythin
#     out = {
#         'input_r': _input_r,
#         'fc1': _fc1,
#         'fc_dr1': _fc_dr1,
#         'out': _out
#     }
    
#     return out

In [10]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, img_y, img_x, img_color])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

_pred = conv_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

_corr = tf.equal(tf.argmax(_pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(_corr, tf.float32))

init = tf.initialize_all_variables()

# Saver
save_step = 1
savedir = 'nets_VGG_test/'
saver = tf.train.Saver(max_to_keep=training_epochs)

In [11]:
!mkdir nets_VGG_test

In [12]:
do_train = 1

# Do some optimizations
sess = tf.Session()

In [13]:
sess.run(init)

In [14]:
if do_train == 1:
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(data_img_name)/batch_size)
        
        # Loop over all batchs
        for i in range(total_batch):
            randidx = np.random.randint(len(data_img_name), size=batch_size)
            
            batch_xs, batch_ys = getTrainData(randidx, data_img_name, data_label)
#             batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # Fit training using batch data
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio: .7})
            
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio: 1.})/batch_xs.shape[0]
            
            if i%10 == 0:
                print (">>> Batch turn: " + str(i) + ", avg_cost: " + str(avg_cost))
            
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, keepratio: 1.})
            print ("Training accuracy: %.3f" % (train_acc))

        # Save Net
        if epoch % save_step == 0:
            saver.save(sess, savedir + 'cnn_1_fcn.ckpt-' + str(epoch))
    
    print ('Optimization Finished.')

>>> Batch turn: 0, avg_cost: 3.82374969482
>>> Batch turn: 10, avg_cost: 64.5908514404
>>> Batch turn: 20, avg_cost: 122.860076904
>>> Batch turn: 30, avg_cost: 173.809180298
>>> Batch turn: 40, avg_cost: 218.965236816
>>> Batch turn: 50, avg_cost: 258.355927124
>>> Batch turn: 60, avg_cost: 294.726842957
>>> Batch turn: 70, avg_cost: 327.905557251
>>> Batch turn: 80, avg_cost: 361.843382874
>>> Batch turn: 90, avg_cost: 394.056391907
>>> Batch turn: 100, avg_cost: 422.695877228
>>> Batch turn: 110, avg_cost: 452.562248993
>>> Batch turn: 120, avg_cost: 482.600773163
>>> Batch turn: 130, avg_cost: 510.648816681
>>> Batch turn: 140, avg_cost: 536.65723053
>>> Batch turn: 150, avg_cost: 563.639736786
>>> Batch turn: 160, avg_cost: 588.800364532
>>> Batch turn: 170, avg_cost: 612.067735443
>>> Batch turn: 180, avg_cost: 635.588895416
>>> Batch turn: 190, avg_cost: 659.564342041
>>> Batch turn: 200, avg_cost: 681.742223511
>>> Batch turn: 210, avg_cost: 706.539822083
>>> Batch turn: 220, a

In [17]:
def dataTestList(pathName):
    data_img_name = []

    for file2 in os.listdir(pathName):
        data_img_name.append(pathName + os.sep + file2)
            
    return data_img_name

In [18]:
test_root_path = "/data/nv_dlcontest_dataset/test_VGG"

test_data_img_name = dataTestList(test_root_path)

In [15]:
def getTestData(idxList, data_img_name):
    num_data = len(idxList)
    _img = np.ndarray(shape=(num_data, img_x, img_y, img_color), dtype=float)
    
    row = 0
    for idx in idxList:

        # file loading
        tmp_img = None
        tmp_fname = data_img_name[idx]
        with open(tmp_fname) as f:
            tmp_img = np.load(f)
            
        if len(tmp_img.shape)<3:
            continue
        
        _img[row] = tmp_img

        row += 1
        
    return _img[:row, :]

In [28]:
ret = []
data_len = len(test_data_img_name)

for s, e in zip(range(0, data_len - batch_size, batch_size), range(batch_size, data_len, batch_size)):
    print (str(s) + ' : ' + str(e))

    batch_xs = getTestData(range(s, e), test_data_img_name)
    tmp_pred = sess.run(tf.argmax(_pred, 1), feed_dict={x: batch_xs, keepratio: 1.})
    ret.extend(tmp_pred)

if data_len/batch_size != 0:
    s = data_len - data_len%batch_size
    e = data_len
    print (str(s) + ' : ' + str(e))

    batch_xs = getTestData(range(s, e), test_data_img_name)
    tmp_pred = sess.run(tf.argmax(_pred, 1), feed_dict={x: batch_xs, keepratio: 1.})
    ret.extend(tmp_pred)
    
    print (batch_xs.shape)
    print (_pred.get_shape())
    print (tmp_pred)

0 : 100
100 : 200
200 : 300
300 : 400
400 : 500
500 : 600
600 : 700
700 : 800
800 : 900
900 : 1000
1000 : 1100
1100 : 1200
1200 : 1300
1300 : 1400
1400 : 1500
1500 : 1600
1600 : 1700
1700 : 1800
1800 : 1900
1900 : 2000
2000 : 2100
2100 : 2200
2200 : 2300
2300 : 2400
2400 : 2500
2500 : 2600
2600 : 2700
2700 : 2800
2800 : 2900
2900 : 3000
3000 : 3100
3100 : 3200
3200 : 3300
3300 : 3400
3400 : 3500
3500 : 3600
3600 : 3700
3700 : 3800
3800 : 3900
3900 : 4000
4000 : 4100
4100 : 4200
4200 : 4300
4300 : 4400
4400 : 4500
4500 : 4600
4600 : 4700
4700 : 4800
4800 : 4900
4900 : 5000
5000 : 5100
5100 : 5200
5200 : 5300
5300 : 5400
5400 : 5500
5500 : 5600
5600 : 5700
5700 : 5800
5800 : 5900
5900 : 6000
6000 : 6100
6100 : 6200
6200 : 6300
6300 : 6400
6400 : 6500
6500 : 6600
6600 : 6700
6700 : 6800
6800 : 6900
6900 : 7000
7000 : 7100
7100 : 7200
7200 : 7300
7300 : 7400
7400 : 7500
7500 : 7600
7600 : 7700
7700 : 7800
7800 : 7900
7900 : 8000
8000 : 8100
8100 : 8200
8200 : 8300
8300 : 8400
8400 : 8500
8

In [33]:
with open('mrthinks_0909.csv', 'w') as f:
    for i, _id in enumerate(ret):
        f.write(test_data_img_name[i].split('/')[-1])
        f.write(',')
        f.write(label_list[_id])
        f.write('\n')

In [34]:
!head mrthinks_0909.csv

98491,spaghetti_carbonara
602016,oysters
294445,pad_thai
368014,hot_dog
340666,spaghetti_carbonara
98367,dakbokkeumtang
64060,oysters
767141,pancakes
353444,filet_mignon
465222,macarons


In [29]:
for i, label in enumerate(label_list):
    print(i, label)

(0, 'scallops')
(1, 'onion_rings')
(2, 'chicken_curry')
(3, 'french_onion_soup')
(4, 'dumplings')
(5, 'nachos')
(6, 'tuna_tartare')
(7, 'gnocchi')
(8, 'gyoza')
(9, 'bruschetta')
(10, 'croque_madame')
(11, 'edamame')
(12, 'grilled_salmon')
(13, 'sashimi')
(14, 'spring_rolls')
(15, 'risotto')
(16, 'prime_rib')
(17, 'pajeon')
(18, 'cannoli')
(19, 'soondae')
(20, 'doenjang_chigae')
(21, 'waffles')
(22, 'macarons')
(23, 'foie_gras')
(24, 'dakbokkeumtang')
(25, 'ceviche')
(26, 'cheese_plate')
(27, 'hot_dog')
(28, 'ojingeo_bokkeum')
(29, 'creme_brulee')
(30, 'steak')
(31, 'falafel')
(32, 'lasagna')
(33, 'cup_cakes')
(34, 'spaghetti_bolognese')
(35, 'omelette')
(36, 'filet_mignon')
(37, 'soondubu_jjigae')
(38, 'chocolate_mousse')
(39, 'caesar_salad')
(40, 'spaghetti_carbonara')
(41, 'peking_duck')
(42, 'macaroni_and_cheese')
(43, 'pulled_pork_sandwich')
(44, 'escargots')
(45, 'chicken_wings')
(46, 'takoyaki')
(47, 'bulgogi')
(48, 'donuts')
(49, 'samosa')
(50, 'miso_soup')
(51, 'panna_cotta')
(

In [14]:
sess.close()